# TD 3 - Divide and Conquer

In [3]:
import numpy as np
import random

# 1. Dominating Element

We start by defining a function for each of the following tasks:
- count the number of occurrences of an element in a table
- check whether a given value is dominant in a table
- compute the dominant element of a table if it exists


In [24]:
def countOccurrences(t, T):
    """
        input  : value t, list or iterator T
        return : number of occurrences of t in T
    """
    return sum(1 for e in T if e == t)

def isDominant(t,T):
    """
        input   : value t, list T
        returns : True if t is a dominant element in T, False otherwise
    """
    return countOccurrences(t, T) > len(T) / 2

def dominant(T):
    """
        input   : list T
        returns : the (unique) dominant element in T or None if no such element exists
    """
    return next((e for e in T[:(len(T)+1)//2] if isDominant(e, T)), None)


Note that the next value of a generator is computed only when needed. This is known as "lazy evaluation". The algorithm is quite naive and has worst case complexity of $O(n^2/2)$. Can you give a worst-case instance ?

In [28]:
# create a table of size Tsize filled randomly with 0 and 1
Tsize = 8 
T = [random.randint(0, 1) for _ in range(Tsize)]

print("Random table T:\n",T,"\n")

# check whether 1 is dominant and print the dominant element
print("Number of 1s in T:", countOccurrences(1, T))
print("\nIs 1 dominant?:", isDominant(1, T))

print("\nDominant term:",dominant(T))

Random table T:
 [1, 1, 0, 0, 0, 0, 0, 0] 

Number of 1s in T: 2

Is 1 dominant?: False

Dominant term: 0


## 1.1. Divide and Conquer

We use here a recursive bisection approach using the fact that a dominant value of <tt>T</tt> must dominate one the two lists obtained by cutting <tt>T</tt> in the middle.


In [31]:
def divideAndConquer(T):
    """
        input   : list T
        returns : the dominant element in T if it exists, None otherwise
    """
    if len(T)==1:
        return T[0]
    center= len(T)//2
    left  = divideAndConquer(T[:center])
    right = divideAndConquer(T[center:])
    if left == right:
        return left
    if isDominant(right,T):
        return right
    if isDominant(left,T):
        return left
    return None
    

In [37]:
# create a table of size Tsize filled randomly with 0, 1, 2
Tsize = 10 
T = [random.randint(0, 2) for _ in range(Tsize)]

# print info and check for a dominant element
print("Random table T:\n",T,"\n")
print("Number of 0s : {}, 1s : {}, 2s : {}".format(T.count(0), T.count(1), T.count(2)))

print("Dominant term by divide and conquer: ",divideAndConquer(T))

Random table T:
 [2, 0, 1, 2, 2, 2, 2, 2, 0, 2] 

Number of 0s : 2, 1s : 1, 2s : 7
Dominant term by divide and conquer:  2


The complexity is here such that $$C(n)=2C(n/2)+2n$$ in the worst case. By the master theorem, this gives: 
$$C(n)=O(n\log n)$$

## 1.2. Variation

If we compare the values of <tt>T</tt> by successive pairs, removing those which have different values and only keeping one of the two in case of equality, starting from $n$ values, a maximum of $n/2$ is maintained (the case of all pairwise equalities).

To confirm that a dominant element <tt>e</tt> of number $k$ of occurrences in <tt>T</tt> (i.e., $k>n/2$, or more accurately $k\geq n/2-1/2$ in both odd and even $n$'s) is still dominant after this process, consider a given pair:

 * if it does not contain <tt>e</tt>, $n$ is decreased but not $k$
 * if it contains <tt>e</tt> and another value, then $n\to n-2$ and $k\to k-1$ and of course $k-1>(n-2)/2=n/2-1$
 * if it contains twice <tt>e</tt>, then $n\to n-1$ and $k\to k-1$, and $k-1\geq (n-1)/2-1/2=n/2-1$, that is $k\geq n/2\geq n/2-1/2$ both for even and off $n$'s.
 
We obtain the following algorithm:

In [40]:
def divideAndConquerPairs(T):
    """
        input   : list T
        returns : dominant item in T or None if it exists and None otherwise
    """
    if len(T)==0:
        return None
    if len(T)==1:
        return T[0]
    if len(T)%2 == 1 and isDominant(T[-1],T):   # in the odd case, last one could be "tipping" the dominance
        return T[-1]        
    
    halfT = [ t1 for t1,t2 in zip(T[::2],T[1::2]) if t1==t2 ]
    candidate = divideAndConquerPairs(halfT)
    if isDominant(candidate,T):
        return candidate

In [42]:
# create a table of size Tsize filled randomly with 0, 1, 2
Tsize = 10 
T = [random.randint(0, 2) for _ in range(Tsize)]

# print info and check for a dominant element
print("Random table T:\n",T,"\n")
print("Number of 0s : {}, 1s : {}, 2s : {}".format(T.count(0), T.count(1), T.count(2)))

print("Dominant term by divide and conquer: ",divideAndConquerPairs(T))

Random table T:
 [2, 1, 1, 2, 0, 1, 0, 1, 0, 2] 

Number of 0s : 3, 1s : 4, 2s : 3
Dominant term by divide and conquer:  None


For this code, say in the even $n$ case, $C(n)=C(n/2)+n$ so that, by the master theorem,
$$C(n)=O(n)$$

# 2. Multiplying large integers – The Karatsuba algorithm

## 2.1. Naive algorithm

To compute the product of two large integers $A$ and $B$, an idea is to use the binary representation of each as $A=A_1A_2$ and $B=B_1B_2$ on $2\times n/2$ bits and see that
$$AB = (A_1B_1)\cdot 2^n + (A_1B_2+A_2B_1)\cdot 2^{n/2}+A_2B_2$$
which reduces the calculations to the products $A_1B_1$, $A_1B_2$, $A_2B_1$ and $A_2B_2$, since the products by powers of two are just bit shifts, which we assume can be performed in constant time.


The cost $C(n)$ for multiplying two $n$-bit numbers is then $C(n)=4\cdot C(n/2)+O(1)$, that is (by the master theorem):
$$C(n)=O(n^2).$$

In [113]:
def toBinaryString(i):
    """
        converts the given number i to a binary string
    """
    return bin(i)[2:]

def naiveProduct(A, B):
    """
        Naive multiplication algorithm
        input   : binary strings A, B
        returns : the product AB of the two numbers as a binary string
    """
    len_A = len(A)
    len_B = len(B)
    
    if min(len_A, len_B) == 0:
        return '0'
    
    # base case : a contains a single digit
    if len_A == 1:
        return B if A[0] == '1' else '0'

    # base case : b contains a single digit    
    if len_B == 1:
        return A if B[0] == '1' else '0'
    
    center_A = len_A // 2
    center_B = len_B // 2

    #be careful to properly fill with zeros; convert to int to perform addition, then back to a binary string
    return toBinaryString(int(naiveProduct(A[center_A:], B[center_B:]), 2) +\
               int(naiveProduct(A[:center_A], B[:center_B]) + ('0' * (len_A - center_A + len_B - center_B)), 2) +\
               int(naiveProduct(A[center_A:], B[:center_B]) + ('0' * (len_B - center_B)), 2) +\
               int(naiveProduct(A[:center_A], B[center_B:]) + ('0' * (len_A - center_A)), 2))
    

In [114]:
A, B = 45, 14

print(naiveProduct(toBinaryString(A), toBinaryString(B)))
print(toBinaryString(A*B))

1001110110
1001110110


## 2.2. Karatsuba

Here we observe that 
$$(A_1 + A_2)(B_1 + B_2) = A_1 B_1 + A_2 B_2 + (A_1 B_2 + A_2 B_1 )$$
so that $A_1 B_2 + A_2 B_1$ can be computed as $(A_1 + A_2)(B_1 + B_2) - A_1 B_1 - A_2 B_2$, which reduces the number of products to compute from $4$ to $3$.

The resulting cost $C(n)$ of multiplying two n-bit numbers is
$$ C(n) = 3C(n/2)+O(1) $$
or equivalently
$$ C(n) = O(n^{\log_2(3)})$$


In [109]:
def karatsuba(A,B):
    """
        Karatsuba's multiplication algorithm
        input   : binary strings A, B
        returns : product AB as a binary string
    """
    # fill with zeros if necessary
    l = max(len(A), len(B))
    A = A.zfill(l)
    B = B.zfill(l)

    # base cases
    if l == 0:
        return '0'
    
    if l == 1:
        return B if A[0] == '1' else '0'

    center = l // 2
    
    P1 = naiveProduct(A[center:], B[center:])
    P2 = naiveProduct(A[:center], B[:center])
    P3 = naiveProduct(bin(int(A[center:], 2) + int(A[:center], 2))[2:], \
                      bin(int(B[center:], 2) + int(B[:center], 2))[2:])
    
    return toBinaryString(int(P1, 2) +\
               int(P2 + ('0' * (2*(l-center))), 2) +\
               int(P3 + ('0' * (l-center)), 2) - int(P1 + ('0' * (l-center)), 2) - int(P2 + ('0' * (l-center)), 2))

In [110]:
A, B = 199, 141

print(karatsuba(bin(A)[2:], bin(B)[2:]))
print(bin(A*B)[2:])

110110110011011
110110110011011


## EXTRAS

### Reminder of the Master Theorem

If
$$C(n)=a C\left({\frac {n}{b}}\right)+O(n^{d}) $$
then,
  * if $d < \log_b a$, $$C(n)=O(n^{\log _{b}a})$$
  * if $d = \log_b ⁡a$, $$C(n)=O(n^d \log ⁡n )$$
  * if $d > \log_b ⁡a$, $$C(n)=O(n^d).$$ 